In [11]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew


In [12]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")


In [13]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']


In [14]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool


In [15]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"


In [16]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)


2024-11-01 17:09:37,813 - 8382724160 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [17]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})


# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: Thought:  First, I need to gather the latest information on the US inflation rate and its impact on the stock market in 2024.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest news on US inflation rate 2024 stock market impact\"}"
## Tool Output: 

Search results: Title: US consumers keep economy on solid ground ahead of elec

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.23it/s]




# Agent: Chief Data Strategist
## Thought: Thought: I now have enough information to analyze market data and trends related to Inflation in the US and its impact on the stock market in 2024.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"US inflation 2024 stock market impact\", \"website\": \"https://www.investopedia.com/\"}"
## Tool Output: 
Relevant Content:
Julius Mansa MFin Meet the Financial Review Board Latest Articles Exxon and Chevron Report Sluggish Profits Nov 01, 2024 What We Learned About AI From This Week's Big Tech Earnings Nov 01, 2024 Peloton Stock Price Levels to Watch After Upbeat Earnings, Naming of New CEO Nov 01, 2024 Cardinal Health Stock Hits All-Time High as Existing Customers Power Results Nov 01, 2024 What to Expect From Warren Buffett’s Berkshire Hathaway Earnings Saturday Nov 01, 2024 Companies Owned by PepsiCo Updated Nov 01, 2024 6 Companies Owned by Bank of America Updated Nov 01, 2024 Will Hurricanes Mar Biden's Econo

In [18]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)


Twitter
🌐 "Is your portfolio ready for inflation in 2024?
Diversifying and focusing on dividends could be
your game-changer. Read more on our latest
insights! [link] #InvestSmart #Inflation"
--------------------------------------------------
LinkedIn
📈 "Inflation impacts the stock market in complex
ways. Discover our expert advice to navigate
inflation in 2024 and make informed investment
decisions. [link] #FinancialInsights #StockMarket"
--------------------------------------------------


In [19]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))


# Understanding Inflation in the US and Its Impact on the Stock Market in 2024

## Introduction to Inflation

Inflation, defined as the rate at which the general level of prices for goods and services rises, erodes purchasing power and can significantly affect economic stability. In 2024, the United States is poised to navigate through fluctuating inflation rates influenced by various external and internal factors, including energy prices, supply chain disruptions, and consumer behavior shifts. It’s essential for investors to grasp the intricacies of inflation, as these factors can lead to broader economic consequences.

### Current State of US Inflation

As of late 2023, the Federal Reserve has been monitoring inflation rates closely, having implemented several interest rate hikes in response to rising consumer prices. According to the Consumer Price Index (CPI), inflation rates held at an average of 3.5% for the year. Forecasts suggest a slight reduction to about 3.0% in 2024, presenting a mixed bag of opportunities and risks. Understanding these trends can help investors make informed decisions about their portfolios, especially amid a potentially volatile stock market.

## Impact of Inflation on the Stock Market

### Historical Perspectives

Historically, inflation impacts stocks unevenly. While certain sectors like utilities and consumer staples tend to perform well during inflationary periods, growth stocks, particularly in tech, may struggle with rising costs and reduced consumer spending capabilities. As we move into 2024, the expectation is that market performance may be more subdued due to sustained inflationary pressures, causing some investors to pivot towards defensive stock options.

### Recommendations for Investors

Investors must adapt their strategies to account for the anticipated inflation levels. Here are several key recommendations:

- **Diversify Portfolios**: Incorporate a mix of asset classes, including commodities and bonds, which tend to hedge against inflation. 
- **Focus on Dividends**: Companies with strong, consistent dividends may provide a buffer against market volatility and inflation impacts.
- **Monitor Sector Performance**: Pay close attention to sectors that traditionally benefit during inflationary periods, such as real estate and materials.

For a deeper dive into asset allocation strategies during inflationary environments, check out our full report [here](#).

## Conclusion

The landscape of the US economy in 2024 will be shaped significantly by inflation and its intricate relationship with the stock market. Staying informed and agile is crucial for investors looking to navigate this challenging environment. As inflation continues to evolve, adapting investment strategies accordingly will be vital for long-term success. 

Constant vigilance in portfolio management is key, coupled with thorough research on market conditions and sector performance. By incorporating the strategies outlined and understanding the implications of inflation, investors can position themselves more favorably in a post-inflation landscape.

---